# TCC - Analise de Bitcoin com base em dados historicos

### Alunos - Ciro Mora e Renan Renger


## Objetivos

### Responder questões pertinentes aos dados obtidos

### Identificar os dados para utilizar num treinamento de rede LSTM

### Treinar uma rede LSTM para gerar o valor medio do preço do bitcoin do dia seguinte com base em dados historicos

# Descrição do dataset
## Terminologia
**Criptoativos /criptomoeda** -> "Moeda digital" baseada em operações matemáticas e com forte apelo para descentralização dos recursos financeiros e maior agilidade, privacidade e segurança do que as alternativas atuais;

**BTC** -> Bitcoin, primeira criptomoeda desenvolvida e lançada comercialmente para negociação, considerada a mais forte e estável dentre as existêntes atualmente devido a sua "resiliência" a crises e desvalorizações; 

**USD** -> United States Dollar ou Dolar Americano, moeda usada como referência para as transações na maioria das bolsas, devido a fácil conversão entre BTC e USD;

**Bitstamp** -> Bolsa de criptoativos online responsável por facilitar as negociações entre traders; Provem informações relacionadas ao valor e as transações efetuadas, servindo como base de dados para esse estudo;

**Trader** -> Negociador que opera no mercado (neste caso, de criptoativos) visando, através de consecutivas operações de compra e venda, obter lucro;

**Long** -> Investimento visando/"apostando" lucro em caso de aumento no valor do ativo, normalmente envolvendo a compra a valores mais baixos e venda a valores mais altos;

**Short** -> Oposto ao Long, visa ganhos "alugando" ativos a um valor mais alto, vendendo os mesmos imediatamente e os recomprando a um preço mais baixo, devolvendo a mesma quantidade de ativos (que valem menos do que quanto "alugados") e lucrando com a diferença entre o valor no momento da venda e da recompra;

## Colunas
**Timestamp** - > Data (em formato Epoch Unix) da coleta dos dados; Será transformada posteriormente em data "humana" para melhor compreensão; Intervalos de aproximadamente 1 em 1 minuto, com fuso definido para UTC;

**Open** -> Valor inicial de negociação da moeda naquele intervalo de medição, em USD;

**High** -> Maior valor atingido pelo ativo durante aquele intervalo de medição, em USD;

**Low** -> Menor valor atingido pelo ativo durante aquele intervalo de medição, em USD;

**Close** -> Valor do ativo no momento de fechamento do intervalo de medição, em USD;

**Volume_(BTC)** -> Volume, em BTC, negociado na Bitstamp durante um dado intervalo de medição;

**Volume_(Currency)** -> Volume, em USD, negociado na Bitstamp durante um dado intervalo de medição;

**Weighted_Price** -> Preço médio do ativo naquele intervalo, em USD; Calculado com base nos volumes negociados; Será considerado como preço médio para questões analiticas.

# Questões Pertinentes
## 1 - Comportamento Temporal
Antes de buscarmos nos aprofundar no dataset, vale uma compreensão do comportamento temporal do ativo, partindo por uma visão cronológica linear e seguindo para uma analise grupos temporais semelhantes, como meses, semanas e dias da semana;

## 2 - Relação entre volume negociado e valor
Para validar uma crença informal (que quanto mais BTC se negocia, maior o valor da moeda), podemos cruzar essas duas variáveis em uma escala temporal e ver existe realmente uma relação entre elas;

## 3 - Volatilidade do Ativo
Uma da variáveis que define a classe de risco de um ativo é seu grau de volatilidade. Antes de seguimos com nossos planos de investir em BTC, devemos analisar qual o grau de risco da moeda;

## 4 - Rentabilidade
Além do risco, para definirmos se um investimento é viável, devemos ver sua rentabilidade histórica (lembrando sempre que isso não nos garante rendimentos futuros e sim deve ser utilizado como base) e cruzar isso com a volatilidade, definindo se desejamos investir ou não em tal ativo;

## 5 - Aptidão a "Day Trade"
O conceito de "Day Trade" é operar em intervalos curtos de tempo, no caso da nossa analise sendo definido como operações no mesmo dia; Para tal, o ativo deve apresentar uma amplitude considerável em seus valores máximos e mínimos, permitindo que a um bom trader (que a analise assume que somos) maximizar seus lucros;

# Pré-processamento dos dados

## 1ºpasso - Conversão de labels de colunas para termos locais

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import os
from datetime import tzinfo, timedelta, datetime
from dateutil import relativedelta
import math

In [ ]:
dfBitstamp = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2019-08-12.csv', delimiter=',')
#dfBitstamp = pd.read_csv('../input/bitcoin-historical-data/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv')
dfBitstamp.dataframeName = 'Bitstamp_USD-BTC_1-min_data'
nRow, nCol = dfBitstamp.shape
print(f'{nRow} linhas por {nCol} colunas')

In [ ]:
dfBitstamp.rename(index=str, columns={
    'Timestamp':'DataHora',
    'Open':'Valor_Abertura',
    'High':'Valor_Maxima',
    'Low':'Valor_Minima',
    'Close':'Valor_Fechamento',
    'Volume_(BTC)':'BTC_Negociado',
    'Volume_(Currency)':'USD_Negociado',
    'Weighted_Price':'Preco_Medio_BTC'
}, inplace=True);

In [ ]:
dfBitstamp.head(15)

In [ ]:
dfBitstamp.info()

## 2º Passo - Conversão da Data Unix para Data "Humana"

In [ ]:
dfBitstamp.DataHora = pd.to_datetime(dfBitstamp.DataHora, unit='s')

In [ ]:
dfBitstamp.head(15)

# Estudo preliminar - Intervalos de data e valor

Antes de seguirmos com a definição de um período para analise, vamos estudar os intervalos em que os dados se encontram, para termos uma visão melhor do todo

In [ ]:
intervaloData = relativedelta.relativedelta(dfBitstamp.DataHora.max(), dfBitstamp.DataHora.min())
print(f'Dataset engloba dados de {intervaloData.years} anos, {intervaloData.months} meses e {intervaloData.days} dias')

# Analises
## 1 - Comportamento Temporal

Vamos iniciar a analise com o comportamente temporal do bitcoin no decorer da sua vida até a data mais atual do dataset.

Gerados dataframes relativos aos intervalos desejados para analise temporal - Anual, Mensal. Semanal e Diario

In [ ]:
dfBitstampPorAno = dfBitstamp.groupby(dfBitstamp.DataHora.dt.year).mean()
dfBitstampPorPeriodo = dfBitstamp.groupby([dfBitstamp.DataHora.dt.year, dfBitstamp.DataHora.dt.month]).mean()
dfBitstampPorMes = dfBitstamp.groupby(dfBitstamp.DataHora.dt.month).mean()
dfBitstampPorDia = dfBitstamp.groupby(dfBitstamp.DataHora.dt.date).mean()
dfBitstampPorSemanaAno = dfBitstamp.groupby(dfBitstamp.DataHora.dt.weekofyear).mean()
dfBitstampPorDiaSemana = dfBitstamp.groupby(dfBitstamp.DataHora.dt.weekday).mean()
dfBitstampPorAno.index.names = dfBitstampPorMes.index.names = dfBitstampPorSemanaAno.index.names = dfBitstampPorDiaSemana.index.names = ['Periodo']

In [ ]:
dfTransacoesValidas = dfBitstamp[dfBitstamp.BTC_Negociado.notna()].DataHora
dfBitstampPorAno['Transacoes_Periodo'] = dfTransacoesValidas.groupby(dfBitstamp.DataHora.dt.year).count()
dfBitstampPorPeriodo['Transacoes_Periodo'] = dfTransacoesValidas.groupby([dfBitstamp.DataHora.dt.year, dfBitstamp.DataHora.dt.month]).count()
dfBitstampPorMes['Transacoes_Periodo'] = dfTransacoesValidas.groupby(dfBitstamp.DataHora.dt.month).count()
dfBitstampPorSemanaAno['Transacoes_Periodo'] = dfTransacoesValidas.groupby(dfBitstamp.DataHora.dt.weekofyear).count()
dfBitstampPorDiaSemana['Transacoes_Periodo'] = dfTransacoesValidas.groupby(dfBitstamp.DataHora.dt.weekday).count()

# Antes de seguirmos, observações devem ser feitas: 

Por questão de simplicidade, estamos assumindo que cada minuto onde houveram transações, UMA E APENAS UMA TRANSAÇÃO FOI FEITA.
Isto é feito para não ser necessária a incrementação do dataset com dados das operações em si (que podem ou não serem fornecidos pela bolsa de criptoativos), atigindo assim uma escala e comportamento mais simplificados para nossa análise.

Vale ainda mencionar que os valores em BTC são exibidos com 9 casas por se tratar da menor unidade tecnicamente negociável (Satoshi, 10^-9 BTC)

## Novamente: Os valores de transações podem (e, geralmente, devem) ser maiores do que os apresentados nos gráficos, os desmonstrados servindo apenas para analise de tendência!

## Analise Temporal - Anual, mensal, semanal e diaria
Para todas as analises temporais, vamos focar no volume de bitcoins negociados em um dado período em virtude da quantidade de transações (**vide a observação acima**) neste mesmo intervalo de tempo;

In [ ]:
traceBTC = go.Bar(
                x = dfBitstampPorAno.index,
                y = np.round(dfBitstampPorAno.BTC_Negociado, 9),
                name = 'BTC/transação',
                marker = dict(color = 'rgba(84, 92, 229, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)),
                text = dfBitstampPorAno.index)

traceActiveMinutes = go.Scatter(
                x = dfBitstampPorAno.index,
                y = dfBitstampPorAno.Transacoes_Periodo,
                text = dfBitstampPorAno.index,
                name = 'Transações ',
                marker = dict(color = 'rgba(168, 69, 227, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)),
                yaxis='y2')

layout = go.Layout(
    title='Anual - Volume negociado x Quantidade de transações',
    xaxis=dict(
        title='Período',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Volume negociado (em BTC)',
        ticksuffix = ' BTC',
        hoverformat = ',9f',
        titlefont=dict(
            size=16
        )
    ),
    yaxis2=dict(
        title='Transações por período',
        ticksuffix = ' transações/periodo',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        ),
        overlaying = 'y',
        side = 'right',
        rangemode = 'tozero'
    )
)

py.iplot(go.Figure(data = [traceBTC, traceActiveMinutes], layout=layout))

### Analise Preliminar
Como era esperado, uma vez que a adoção do Bitcoin como ativo negociável começou a "engrenar" em 2013, houve uma regularidade no volume de transações quebrada apenas por 2017, ano de maior visibilidade do ativo na mídia. 
No que tange ao volume por transação, 2015 e 2016 apresentam comportamentos anômalos, com variações muito distintas do padrão dos outros anos (aumento na quantidade de transações, diminuição do volume transferido a cada operação) 

In [ ]:
traceBTC = go.Bar(
                x = dfBitstampPorMes.index,
                y = np.round(dfBitstampPorMes.BTC_Negociado, 9),
                name = 'BTC/transação',
                marker = dict(color = 'rgba(84, 92, 229, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)),
                text = ['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro'])

traceActiveMinutes = go.Scatter(
                x = dfBitstampPorMes.index,
                y = dfBitstampPorMes.Transacoes_Periodo,
                text = ['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro'],
                name = 'Transações ',
                marker = dict(color = 'rgba(168, 69, 227, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)),
                yaxis='y2')

layout = go.Layout(
    title='Mensal - Volume negociado x Quantidade de transações',
    xaxis=dict(
        title='Período',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Volume negociado (em BTC)',
        ticksuffix = ' BTC',
        hoverformat = ',9f',
        titlefont=dict(
            size=16
        )
    ),
    yaxis2=dict(
        title='Transações por período',
        ticksuffix = ' transações/periodo',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        ),
        overlaying = 'y',
        side = 'right',
        rangemode = 'tozero'
    )
)

py.iplot(go.Figure(data = [traceBTC, traceActiveMinutes], layout=layout))

### Analise Preliminar
Enquanto a quantidade de transações por mês mantem uma regularidade, o valor por transação apresenta uma queda nos meses centrais, com altas no ínicio e fim do período.
Talvez ao cruzar com os dados de valor médio da moeda nesses meses, possamos compreender o que causou tal comportamento.

In [ ]:
traceBTC = go.Bar(
                x = dfBitstampPorSemanaAno.index,
                y = np.round(dfBitstampPorSemanaAno.BTC_Negociado, 9),
                name = 'BTC/transação',
                marker = dict(color = 'rgba(84, 92, 229, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)),
                text = dfBitstampPorSemanaAno.index)

traceActiveMinutes = go.Scatter(
                x = dfBitstampPorSemanaAno.index,
                y = dfBitstampPorSemanaAno.Transacoes_Periodo,
                text = dfBitstampPorSemanaAno.index,
                name = 'Transações ',
                marker = dict(color = 'rgba(168, 69, 227, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)),
                yaxis='y2')

layout = go.Layout(
    title='Semanal - Volume negociado x Quantidade de transações',
    xaxis=dict(
        title='Período',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Volume negociado (em BTC)',
        ticksuffix = ' BTC',
        hoverformat = ',9f',
        titlefont=dict(
            size=16
        )
    ),
    yaxis2=dict(
        title='Transações por período',
        ticksuffix = ' transações/periodo',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        ),
        overlaying = 'y',
        side = 'right',
        rangemode = 'tozero'
    )
)

py.iplot(go.Figure(data = [traceBTC, traceActiveMinutes], layout=layout))

### Analise Preliminar
Assim como na escala mensal (e como era esperado), a visão semanal nos mostra uma regularidade na quantidade de transações e uma maior estaabilidade, com alguns movimentos indicando possível padrão, surgindo mais claramente na semana 14 e seguindo, com algumas pequenas variações, até a semana 39;
Para um trader, esse poderia ser um momento de atenção e maior analise, pois pode revelar uma queda mais acentuada no preço e maior potencial de retorno (iremos explorar essa possibilidade em momento oportuno);
Uma observação é merecida ao período que engloba as semanas 51, 52 e 53. Podemos ver claramente uma queda brusca tanto na quantidade de transações quanto no volume das mesmas nessas semanas, fazendo valer uma analise mais detalhada em momento futuro;

In [ ]:
traceBTC = go.Bar(
                x = dfBitstampPorDiaSemana.index,
                y = np.round(dfBitstampPorDiaSemana.BTC_Negociado, 9),
                name = 'BTC/transação',
                marker = dict(color = 'rgba(84, 92, 229, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)),
                text = ['Domingo','Segunda','Terça','Quarta','Quinta','Sexta','Sábado'])

traceActiveMinutes = go.Scatter(
                x = dfBitstampPorDiaSemana.index,
                y = dfBitstampPorDiaSemana.Transacoes_Periodo,
                text = ['Domingo','Segunda','Terça','Quarta','Quinta','Sexta','Sábado'],
                name = 'Transações ',
                marker = dict(color = 'rgba(168, 69, 227, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)),
                yaxis='y2')

layout = go.Layout(
    title='Diario - Volume negociado x Quantidade de transações',
    xaxis=dict(
        title='Período',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Volume negociado (em BTC)',
        ticksuffix = ' BTC',
        hoverformat = ',9f',
        titlefont=dict(
            size=16
        )
    ),
    yaxis2=dict(
        title='Transações por período',
        ticksuffix = ' transações/periodo',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        ),
        overlaying = 'y',
        side = 'right',
        rangemode = 'tozero'
    )
)

py.iplot(go.Figure(data = [traceBTC, traceActiveMinutes], layout=layout))

### Analise Preliminar
Salvo por uma queda na sexta e sábado, que pode ou não estar associada a uma questão de mercado com maior fluxo (China) e seu fuso horário, existe uma consistência na quantidade de transações por dia, assim como um padrão de transações mais "volumosas" no meio da semana;

## 2 - Relação entre volume negociado e valor

In [ ]:
dfBitstampPorPeriodo.index.names = ['Ano','Mes']
dfBitstampPorPeriodo.index = [dfBitstampPorPeriodo.index.get_level_values(0), dfBitstampPorPeriodo.index.map('{0[1]}/{0[0]}'.format)]
dfBitstampPorPeriodo.index.names = ['Ano','Periodo']

In [ ]:
tracePorcentagem = go.Scatter(
                name = 'Volume/período',
                x = dfBitstampPorPeriodo.index.get_level_values(1),
                y = np.round(dfBitstampPorPeriodo.BTC_Negociado, 9),
                marker = dict(color = 'rgba(84, 92, 229, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)),
                text = dfBitstampPorPeriodo.index.get_level_values(1))

traceUSD = go.Scatter(
                name = 'Valor médio',
                x = dfBitstampPorPeriodo.index.get_level_values(1),
                y = np.round(dfBitstampPorPeriodo.Preco_Medio_BTC, 2),
                marker = dict(color = 'rgba(229, 84, 165, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)),
                text = dfBitstampPorPeriodo.index.get_level_values(1),
                yaxis='y2')

layout = go.Layout(
    title='Volume negociado (em BTC) x Valor médio (em USD), por período',
    xaxis=dict(
        title='Período (Mês/Ano)',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Volume negociado no período (em BTC)',
        ticksuffix = ' BTC',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        )
    ),
    yaxis2=dict(
        title='Valor médio por BTC (em USD)',
        tickprefix = 'USD ',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        ),
        overlaying = 'y',
        side = 'right'
    )
)

py.iplot(go.Figure(data = [tracePorcentagem, traceUSD], layout=layout))

In [ ]:
df2018 = dfBitstamp[dfBitstamp.DataHora.dt.year >= 2018]
dfBitstampAtualDiario = df2018.groupby(by=dfBitstamp.DataHora.dt.date).mean()

In [ ]:
trace = go.Scatter(
                x = dfBitstampAtualDiario.index,
                y = np.round(dfBitstampAtualDiario.Preco_Medio_BTC, 2),
                marker = dict(color = 'rgba(84, 92, 229, 0.9)', line=dict(color='rgb(0,0,0)',width=1.5)))
layout = go.Layout(
    title='Valor do BTC por Período (Mês/Ano) a partir de 2018',
    xaxis=dict(
        title='Período',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Valor de 1 BTC (em USD)',
        tickprefix = 'USD ',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [trace], layout=layout))

### Analise Preliminar
Infelizmente nenhuma relação (tanto diretamento quanto inversamente) foi observada entre o valor médio de venda e o volume de BTC no período;

Talvez melhorando o conjunto de dados com informações das ordens em si e considerando o quanto desse volume foi de compra e quanto de venda, possamos chegar a uma relação minimamente informativa entre as duas grandezas.

## 3 - Volatilidade do Ativo e 4 - Rentabilidade

Para todas as analises de agora em diante, vamos focar apenas nos dados a partir de 2018, visto que queremos entender o comportamento financeiro do ativo e que, por essa ótica, dados anteriores não representam o comportamento da moeda, como visto no gráfico de comportamento anual;

**Para calculo da Volatilidade, usaremos o desvio padrão da porcentagem de rentabilidade em cada um dos períodos**, calculado da seguinte forma:

**Rentabilidade no Período: (pAtual - pAnterior)/pAnterior**, sendo **pAtual = Preço no Período Corrente** e **pAnterior = Preço no Período Anterior**

---

Considerando a proximidade entre as duas grandezas a serem analisadas (a volatilidade é calculada a partir da rentabilidade, afinal), faremos uma analise conjunta de ambas, tanto em intervalos diarios, quanto mensais e semanais

In [ ]:
dfBitstampAtualSemanal = df2018.groupby(by=dfBitstamp.DataHora.dt.weekofyear).mean()
dfBitstampAtualMensal = df2018.groupby(by=dfBitstamp.DataHora.dt.month).mean()

In [ ]:
dfBitstampAtualDiario['Variacao_Percentual'] = ((dfBitstampAtualDiario.Preco_Medio_BTC - dfBitstampAtualDiario.Preco_Medio_BTC.shift()) / dfBitstampAtualDiario.Preco_Medio_BTC) * 100
dfBitstampAtualDiario.Variacao_Percentual.fillna(0, inplace=True)
dfBitstampAtualDiario.index = pd.to_datetime(dfBitstampAtualDiario.index)
dfBitstampAtualDiario['Acumulado_Variacao'] = dfBitstampAtualDiario.Variacao_Percentual.cumsum()
dfBitstampAtualDiario['Volatilidade'] = np.std(dfBitstampAtualDiario.Variacao_Percentual)

In [ ]:
dfBitstampAtualSemanal['Variacao_Percentual'] = ((dfBitstampAtualSemanal.Preco_Medio_BTC - dfBitstampAtualSemanal.Preco_Medio_BTC.shift()) / dfBitstampAtualSemanal.Preco_Medio_BTC) * 100
dfBitstampAtualSemanal.Variacao_Percentual.fillna(0, inplace=True)
dfBitstampAtualSemanal['Acumulado_Variacao'] = dfBitstampAtualSemanal.Variacao_Percentual.cumsum()
dfBitstampAtualSemanal['Volatilidade'] = np.std(dfBitstampAtualSemanal.Variacao_Percentual)

In [ ]:
dfBitstampAtualMensal['Variacao_Percentual'] = ((dfBitstampAtualMensal.Preco_Medio_BTC - dfBitstampAtualMensal.Preco_Medio_BTC.shift()) / dfBitstampAtualMensal.Preco_Medio_BTC) * 100
dfBitstampAtualMensal.Variacao_Percentual.fillna(0, inplace=True)
dfBitstampAtualMensal['Acumulado_Variacao'] = dfBitstampAtualMensal.Variacao_Percentual.cumsum()
dfBitstampAtualMensal['Volatilidade'] = np.std(dfBitstampAtualMensal.Variacao_Percentual)

In [ ]:
tracePorcentagem = go.Scatter(
                name = 'Rentabilidade Diaria',
                x = dfBitstampAtualDiario.index,
                y = dfBitstampAtualDiario.Variacao_Percentual,
                line = dict(
                    color = 'rgba(84, 92, 229, 0.9)',
                    width = 1.5
                ),
                text = dfBitstampPorPeriodo.index)

traceVolatilidadeMaxima = go.Scatter(
                name = 'Volatilidade - Limite Superior',
                x = dfBitstampAtualDiario.index,
                y = dfBitstampAtualDiario.Volatilidade,
                line = dict(
                    color = 'rgba(229, 84, 165, 0.9)',
                    width = 4,
                    dash = 'dash'
                ),
                text = dfBitstampPorPeriodo.index
)

traceVolatilidadeMinima = go.Scatter(
                name = 'Volatilidade - Limite Inferior',
                x = dfBitstampAtualDiario.index,
                y = - dfBitstampAtualDiario.Volatilidade,
                line = dict(
                    color = 'rgba(229, 220, 84, 0.9)',
                    width = 4,
                    dash = 'dash'
                ),
                text = dfBitstampPorPeriodo.index
)

layout = go.Layout(
    title='Volatilidade x Rentabilidade Diaria, com limites superiores e inferiores',
    xaxis=dict(
        title='Dia',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='%',
        ticksuffix = ' %',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [tracePorcentagem, traceVolatilidadeMaxima, traceVolatilidadeMinima], layout=layout))

## Analise Preliminar

A priori, 3,60% de volatilidade diária é um valor elevadíssimo, muito além do normal de outros investimentos de risco semelhante (tais índices geralmente são atigindos somente com noticias, divulgações de resultados ou comunicados de extrema importância, sendo considerados como eventos anômalos). Para piorar a situação, durante 3 ou 4 primeiros meses do ano de 2018, por termos vindo de um crash da moeda, temos taxas de rentabilidade ainda mais desconexas com a volatilidade calculada, elevando ainda mais o risco de investimentos no ativo.

O que chama a atenção é a eventual estabilização da moeda conforme o ano se transcorria, os efeitos do crash dando lugar a uma rentabilidade e um risco menor, assumindo um comportamento quase que totalmente dentro da volatilidade do período ao chegarmos na segunda metade de Agosto, com apenas um caso de quebra de padrão (06/09), podendo estar associado a algum evento anômalo que pode ser investigado em momento oportuno.

Vale mencionar, ainda assim, que investidores cientes do risco apresentado pelo ativo poderiam lucrar tanto "na alta" (com operações do tipo "long") quanto "na baixa" ("short" em dias de queda acentuada), em especial na primeira semana de Fevereiro, que apresentou taxas muito além das calculadas pro período.

In [ ]:
tracePorcentagem = go.Scatter(
                name = 'Rentabilidade Semanal',
                x = dfBitstampAtualSemanal.index,
                y = dfBitstampAtualSemanal.Variacao_Percentual,
                line = dict(
                    color = 'rgba(84, 92, 229, 0.9)',
                    width = 1.5
                ),
                text = dfBitstampAtualSemanal.index)

traceVolatilidadeMaxima = go.Scatter(
                name = 'Volatilidade - Limite Superior',
                x = dfBitstampAtualSemanal.index,
                y = dfBitstampAtualSemanal.Volatilidade,
                line = dict(
                    color = 'rgba(229, 84, 165, 0.9)', 
                    width = 4,
                    dash = 'dash'
                ),
                text = dfBitstampAtualSemanal.index
)

traceVolatilidadeMinima = go.Scatter(
                name = 'Volatilidade - Limite Inferior',
                x = dfBitstampAtualSemanal.index,
                y = - dfBitstampAtualSemanal.Volatilidade,
                line = dict(
                    color = 'rgba(229, 220, 84, 0.9)',
                    width = 4,
                    dash = 'dash'
                ),
                text = dfBitstampAtualSemanal.index
)

layout = go.Layout(
    title='Volatilidade x Rentabilidade Semanal, com limites superiores e inferiores',
    xaxis=dict(
        title='Semana do ano (1 - 54)',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='%',
        ticksuffix = ' %',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [tracePorcentagem, traceVolatilidadeMaxima, traceVolatilidadeMinima], layout=layout))

## Analise Preliminar

Ainda mantendo o comportamento diario, a visão semanal apresenta todas as mesmas caracteristicas que já esperavamos, com boas oportunidades para "short" e "long" durante o começo do ano e uma maior estabilidade nos valores no final do ano, quando o "hype" já havia morrido e grande parte da mídia deixava de noticiar quase diariamente a situação da criptomoeda.

Enfase para as semanas 24 a 32, onde podemos ver claramente um padrão chamado de "Retração de Fibonacci", onde o ativo salta de valor (24 - > 25), tem uma queda pequena (25 -> 26), uma nova alta (26 -> 27), uma nova baixa (27 -> 28) e, finalmente, uma ultima disparada (28 -> 29) antes de cair para o valor inicial da corrida (29 -> 32).

In [ ]:
tracePorcentagem = go.Scatter(
                name = 'Rentabilidade Mensal',
                x = dfBitstampAtualMensal.index,
                y = dfBitstampAtualMensal.Variacao_Percentual,
                line = dict(
                    color = 'rgba(84, 92, 229, 0.9)',
                    width = 1.5
                ),
                text = dfBitstampAtualMensal.index)

traceVolatilidadeMaxima = go.Scatter(
                name = 'Volatilidade - Limite Superior',
                x = dfBitstampAtualMensal.index,
                y = dfBitstampAtualMensal.Volatilidade,
                line = dict(
                    color = 'rgba(229, 84, 165, 0.9)',
                    width = 4,
                    dash = 'dash'
                ),
                text = dfBitstampAtualMensal.index
)

traceVolatilidadeMinima = go.Scatter(
                name = 'Volatilidade - Limite Inferior',
                x = dfBitstampAtualMensal.index,
                y = - dfBitstampAtualMensal.Volatilidade,
                line = dict(
                    color = 'rgba(229, 220, 84, 0.9)',
                    width = 4,
                    dash = 'dash'
                ),
                text = dfBitstampAtualMensal.index
)

layout = go.Layout(
    title='Volatilidade x Rentabilidade Mensal, com limites superiores e inferiores',
    xaxis=dict(
        title='Mês (1 - 12)',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='%',
        ticksuffix = ' %',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [tracePorcentagem, traceVolatilidadeMaxima, traceVolatilidadeMinima], layout=layout))

## Analise Preliminar

O mais estável dos gráficos, a volatilidade mensal do BTC ultrapassa a anual de opções mais conservadoras, situando-se acima dos 12% a.m.

Mesmo assim, dois meses apresentaram rentabilidade fora desse alcançe (Fevereiro e Junho), com perdas muito superiores ao limite da volatilidade (37% e 24%, respectivamente). 
No que tange a Fevereiro, o comportamento é esperado, visto as discrepâncias no que tange a rentabilidade diaria, cabendo uma analise mais detalhada de Junho para que possamos localizar a causa dessa anômalia.

# Conclusão

Como era esperado, o BTC apresenta uma volatilidade muito além de outras modalidades de investimento de risco semelhantes (salvo por eventos anômalos nessa modalidades), porém com uma rentabilidade igualmente elevada, que vem decaindo lentamente conforma a moeda atinge uma nova fase de letargia após o "boom" ocorrido em 2017.

Para um trader que busca retornos rápidos, o BTC deixou de lado a caracteristica explosiva dele e tornou-se desvantajoso, substituido facilmente por ações ou pelo próprio IBOV, no caso de trader brasileiros.
Já para nossa proposta, o comportamento mais estável da moeda torna mais fácil da automatização do processo, visto que a chance de detectarmos padrões é maior em dados mais centrados.

## 5 - Aptidão a "Day Trade"

Como dito antes, o conceito de "Day Trade" se resume em efetuar operações que são iniciadas e finalizadas (ou "abertas" e "fechadas", na terminologia financeira) no mesmo dia, contabilizando lucro ou prejuizo por dia ao invés de períodos maiores.

**Como possível Day Trader, interessa saber se o bitcoin apresenta o potencial de rentabilidade diaria que considero "justa" pelo seu risco, em torno de 2% ao dia (ou 60% a.m) ou , caso esteja "de bom humor", poderia até mesmo operar com uma margem de "apenas" 1% a.d (30% a.m).**

Caso a moeda não atinja essa margens, trabalharemos apenas com "Swing Trade", operando em escalas semanais e gastando menos "recursos" com o desenvolvimento de um "robô".

In [ ]:
dfBitstampAtualDiario['Ganho_Day_Trade'] = dfBitstampAtualDiario.Valor_Maxima - dfBitstampAtualDiario.Valor_Minima
dfBitstampAtualDiario['Percentual_Day_Trade'] = ((dfBitstampAtualDiario.Valor_Maxima - dfBitstampAtualDiario.Valor_Minima) / dfBitstampAtualDiario.Valor_Minima) * 100

In [ ]:
tracePorcentagem = go.Scatter(
                name = 'Preço Médio (em USD)',
                x = dfBitstampAtualDiario.index,
                y = dfBitstampAtualDiario.Preco_Medio_BTC,
                line = dict(
                    color = 'rgba(84, 92, 229, 0.9)',
                    width = 1.5
                ),
                text = dfBitstampAtualDiario.index)

traceVolatilidadeMaxima = go.Scatter(
                name = 'Preço Máximo (em USD)',
                x = dfBitstampAtualDiario.index,
                y = dfBitstampAtualDiario.Valor_Maxima,
                line = dict(
                    color = 'rgba(229, 84, 165, 0.9)',
                    width = 1.5,
                    dash = 'dash'
                ),
                text = dfBitstampAtualDiario.index
)

traceVolatilidadeMinima = go.Scatter(
                name = 'Preço Minimo (em USD)',
                x = dfBitstampAtualDiario.index,
                y = dfBitstampAtualDiario.Valor_Minima,
                line = dict(
                    color = 'rgba(229, 220, 84, 0.9)',
                    width = 1.5,
                    dash = 'dash'
                ),
                text = dfBitstampAtualDiario.index
)

layout = go.Layout(
    title='Valores Negociados (em USD/BTC) x Ganho Possível (em %/BTC)',
    xaxis=dict(
        title='Dia',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Valor por BTC (em USD)',
        tickprefix = 'USD ',
        titlefont=dict(
            size=16
        )
    )
)

py.iplot(go.Figure(data = [tracePorcentagem, traceVolatilidadeMaxima, traceVolatilidadeMinima], layout=layout))

## Analise Preliminar

A ausência de três curvas no gráfico já me soa como um pessímo sinal para a rentabilidade potencial da moeda.
Antes de tomar uma decisão, no entanto, vou buscar o máximo de lucro potencial que o bitcoin me oferece diariamente.

In [ ]:
traceGanhoUSD = go.Scatter(
                name = 'Ganho Máximo (em USD)',
                x = dfBitstampAtualDiario.index,
                y = round(dfBitstampAtualDiario.Ganho_Day_Trade, 2),
                line = dict(
                    color = 'rgba(84, 92, 229, 0.9)',
                    width = 1.5
                ),
                text = dfBitstampAtualDiario.index)

traceGanhoPorcentagem = go.Scatter(
                name = 'Ganho Máximo (em %)',
                x = dfBitstampAtualDiario.index,
                y = round(dfBitstampAtualDiario.Percentual_Day_Trade, 6),
                line = dict(
                    color = 'rgba(229, 84, 165, 0.9)',
                    width = 1.5,
                    dash = 'dash'
                ),
                text = dfBitstampAtualDiario.index,
                yaxis = 'y2'
)

layout = go.Layout(
    title='Ganho Máximo (em USD/BTC e %/BTC) x Data',
    xaxis=dict(
        title='Dia',
        titlefont=dict(
            size=16
        )
    ),
    yaxis=dict(
        title='Valor por BTC (em USD)',
        tickprefix = 'USD ',
        hoverformat = ',2f',
        titlefont=dict(
            size=16
        )
    ),
    yaxis2=dict(
        title='Percentual por BTC (em %)',
        ticksuffix = ' %',
        hoverformat = ',6f',
        titlefont=dict(
            size=16
        ),
        overlaying = 'y',
        side = 'right'
    )
)

py.iplot(go.Figure(data = [traceGanhoUSD, traceGanhoPorcentagem], layout=layout))

In [ ]:
print(f'Nossa massa de dados engloba {dfBitstampAtualDiario.Percentual_Day_Trade.count()} dias.')
print(f'Temos {dfBitstampAtualDiario[dfBitstampAtualDiario.Percentual_Day_Trade > 0.5].Percentual_Day_Trade.count()} casos onde o lucro potencial diario é acima de 0.5% (apenas por consideração)')
print(f'Temos {dfBitstampAtualDiario[dfBitstampAtualDiario.Percentual_Day_Trade > 1].Percentual_Day_Trade.count()} casos onde o lucro potencial diario é acima de 1% (minimo aceitável)')
print(f'Temos {dfBitstampAtualDiario[dfBitstampAtualDiario.Percentual_Day_Trade > 2].Percentual_Day_Trade.count()} casos onde o lucro potencial diario é acima de 2% (desejado)')
print(f'A média de lucro máximo diario é de {round(dfBitstampAtualDiario.Percentual_Day_Trade.mean(),6)}%, que representa USD {round(dfBitstampAtualDiario.Ganho_Day_Trade.mean(),2)} por BTC negociado')

## Conclusão

Mesmo que desconsiderassemos a taxa de transferencia da rede (que possivelmente comeria o lucro de qualquer transação abaixo de 0.5% de lucro), as margens que podemos conseguir no Day Trade são tão baixas que não valem o esforço de desenvolver um algoritmo que consiga operar nessa escala, valendo mais a pena focar no Swing Trade e obter lucros "reais".

# Identificar os dados para utilizar num treinamento de rede LSTM

In [ ]:
import keras as K
import tensorflow as tf

In [ ]:
dfCasoDeEstudo = df2018
dfBitstampAtualDiario2019 = dfCasoDeEstudo.groupby(by=dfBitstamp.DataHora.dt.date).mean()
dfBitstampPorPeriodo2019 = dfCasoDeEstudo.groupby([dfCasoDeEstudo.DataHora.dt.year, dfCasoDeEstudo.DataHora.dt.month]).mean()

In [ ]:
dfBitstampAtualSemanal2019 = dfCasoDeEstudo.groupby(by=dfBitstamp.DataHora.dt.weekofyear).mean()
dfBitstampAtualMensal2019 = dfCasoDeEstudo.groupby(by=dfBitstamp.DataHora.dt.month).mean()

## Considerações do treinamento de deep learning

Iremos separar as variaveis e organizar uma database para treinamento do valores de bitcoin, para identificar se haverá flutuação de bitcoin, e qual o valor (inclusive com a direção). 

Porém é necessário saber qual resultado queremos obter. Por exemplo, se obtermos o preco medio diario do proximo dia... o que isso nos diz? Se o preco cair, podemos vender, e se o preco subir, podemos comprar? E se fizermos para uma semana? O preço medio diário ser maior ou menor não significa que devemos comprar ou vender imediatamente o investimento feito no bitcoin. 
Portanto, por hora, gostariamos de utilizar uma rede neural com base nos dados passados e tentar bater esses valores com valores já existentes antes de tentar adivinhar o futuro.

No caso, se pegassemos todos os dados de 2019 até julho, e utilizassemos esses dados para verificar os valores medios obtidos dia a dia de agosto, esses teriam que bater. E a partir dessa relação sim, decidir se é valido investir (comprar) ou vender. Porém é possível essa análise não retorne nenhum resultado, ou um resultado tão bom que seja impreciso com a realidade.

Teremos nossas variaveis X (df2019) para entrada de dados, e y para saido de resultado classificado.

In [ ]:
#utilizando os dados de 2018-2019, vamos treinar uma rede. 
daily_up_or_down = np.zeros(dfCasoDeEstudo.Preco_Medio_BTC.size)
count = 0
for i in range(dfCasoDeEstudo.fillna(0).Preco_Medio_BTC.size):
    if i == 0:
        continue
    else:
        if dfCasoDeEstudo.Preco_Medio_BTC[i] >= dfCasoDeEstudo.Preco_Medio_BTC[i-1]:
            daily_up_or_down[i] = 1
        else:
            daily_up_or_down[i] = 0
        if dfCasoDeEstudo.Preco_Medio_BTC[i] == dfCasoDeEstudo.Preco_Medio_BTC[i-1]:
            count+=1

### Faremos uma classificação bem simples com apenas dois valores para nossa variável de verificação Y.

### Se o preço medio de um dia para o outro subiu ou estagnou, será atribuido o valor 1, ou seja, venda.
### caso contrario, 0

o ideal seria ter um terceiro tipo para -1 quando o valor médio não é alterado, mas os casos de estagnacao de precos sao bem raros, aprox 1% dos dados obtidos, então não vale a pena uma classicação só para eles

#### Os dados de treinamento da variavel X (df2019) serão splitados 70-30, utilizando os primeiros 70% dos dados historicamente para tentar adivinhar os ultimos 30%

## Variaveis utlizadas

Utilizaremos somente duas variaveis para entrada, DataHora e PreçoMedio. isso é para simplificar a análise do LSTM que será desenvolvido mais a frente. Caso essas variaveis não apresentem resultado significativo, faremos uma análise mais detalhada e com mais variáveis.

In [ ]:
len70 = dfCasoDeEstudo.Preco_Medio_BTC.size * 70 / 100
maxlen = dfCasoDeEstudo.Preco_Medio_BTC.size

In [ ]:
#split de 70-30
#y sempre deslocado um valor pra baixo
df2019_train = dfCasoDeEstudo.reset_index().drop(['DataHora', 'Valor_Abertura', 'Valor_Maxima', 'Valor_Minima','Valor_Fechamento', 'BTC_Negociado', 'USD_Negociado'], axis=1)[0:int(len70)]
df2019_test = dfCasoDeEstudo.reset_index().drop(['DataHora','Valor_Abertura', 'Valor_Maxima', 'Valor_Minima','Valor_Fechamento', 'BTC_Negociado', 'USD_Negociado'], axis=1)[int(len70):maxlen-1]
y_train = daily_up_or_down[0:int(len70)]
y_test = daily_up_or_down[int(len70):maxlen-1]

print(f"df2019_train shape is {df2019_train.shape}")
print(f"df2019_test shape is {df2019_test.shape}")
print(f"y_train shape is {y_train.shape}")
print(f"y_test shape is {y_test.shape}")


In [ ]:
df2019_train = df2019_train.fillna(0)
df2019_test = df2019_test.fillna(0)

# Treinando uma rede LSTM para gerar o valor medio do preço do bitcoin do dia seguinte com base em dados historicos 

### Iniciando o treinamento com a rede LSTM. 

### Teste de MLP 1 - rede composta por 4 camadas, sendo a primeira nInputs, a segunda 8 nos, a terceira 10 nos, e a camada de saida com apenas um valor, 1 ou 0

Entrada - dataset somente com preço-medio do BTC 
Saida - fizemos o encoding retornar 1 caso o valor do dia seja maior que o do dia anterior, 0 caso contrario.

#### Parametros

In [ ]:
batch_size = 32
time_steps = 1
prediction = 1 #valor a ser previsto (um dia a frente)
num_features = 1 # numero de atributos usados; DataHora, Preco Medio BTC

#### Montando o modelo

In [ ]:
RS_VALUE = True
#Limpa o modelo previamente usado, senão irá acumular camadas
K.backend.clear_session()
tf.logging.set_verbosity(tf.logging.ERROR) #desliga os warnings do tensorflow
init   = K.initializers.glorot_uniform(seed=1)
theOptimizer = K.optimizers.RMSprop()
model = K.models.Sequential()
model.add(K.layers.LSTM(units=32, return_sequences=RS_VALUE,
                        batch_input_shape=(batch_size, time_steps, num_features),
                        kernel_initializer=init, activation='relu'))
model.add(K.layers.LSTM(units=8, return_sequences=RS_VALUE))
model.add(K.layers.LSTM(units=10, return_sequences=RS_VALUE))
model.add(K.layers.Dense(units=1,  kernel_initializer=init, activation='tanh'))
#model.compile(loss='binary_crossentropy', optimizer=theOptimizer, metrics=['mae','mse'])
#model.compile(loss='mean_squared_error', optimizer=theOptimizer, metrics=['mae','mse'])
model.compile(optimizer=theOptimizer,
              loss=tf.keras.metrics.mean_squared_error,
              metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
model.summary()
print(f"batch_size is {batch_size}")
#Train
max_epochs = 100
print("Iniciando treinamento... ")
X = df2019_train.Preco_Medio_BTC[0:].values[0:((df2019_train.Preco_Medio_BTC.size) - (df2019_train.Preco_Medio_BTC.size % time_steps))]
X_train = X.reshape(-1, time_steps, num_features) #Esta formatação é necessária para o LSTM
y = y_train[0:((df2019_train.Preco_Medio_BTC.size) - (df2019_train.Preco_Medio_BTC.size % time_steps))]
y_train = y.reshape(-1, time_steps, num_features)
h = model.fit(X_train, y_train, batch_size=batch_size, shuffle=False, verbose=1)
print("Treinamento finalizado \n")

In [ ]:
X_test = df2019_test.Preco_Medio_BTC[0:].values[0:((df2019_test.Preco_Medio_BTC.size) - (df2019_test.Preco_Medio_BTC.size % time_steps))]
X_test = X_test.reshape(-1, time_steps, num_features)
y_test = y_test[0:((df2019_test.Preco_Medio_BTC.size) - (df2019_test.Preco_Medio_BTC.size % time_steps))]
y_test = y_test.reshape(-1, time_steps, num_features)

### Resultado do treinamento

In [ ]:
eval = model.evaluate(X_test, y_test, verbose=0)
print("Dados de teste: loss = %0.6f MAE = %0.2f%% \n" \
% (eval[0], eval[1]*100) )

In [ ]:
y_pred = model.predict(X_test, batch_size=batch_size, verbose=0)

In [ ]:
y_test_values = y_test.T[0][0][0:]
y_pred_values = y_pred.T[0][0][0:]

In [ ]:
y_pred_values[y_pred_values>0] = 1
y_pred_values[y_pred_values<=0] = 0

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_values, y_pred_values))

#### Segundo treinamento	
Como o treinamento com valores binários mostrou sinais de overfitting, vamos utilizar os valores normais como parâmetros de entrada, e o valor de fechamento como saida. Assim, o valor obtido após treinamento será numérico, e possivelmente com mais features teremos outra análise do sistema.

In [ ]:
#split de 70-30
#y esta sendo deslocado um valor pra baixo, para simular o dia seguinte
df2019_train2 = dfCasoDeEstudo.reset_index().drop(['index','DataHora','Valor_Fechamento', 'BTC_Negociado', 'USD_Negociado'], axis=1)[0:int(len70)]
df2019_test2 = dfCasoDeEstudo.reset_index().drop(['index','DataHora','Valor_Fechamento', 'BTC_Negociado', 'USD_Negociado'], axis=1)[int(len70):maxlen-1]
y_train2 = dfCasoDeEstudo.fillna(0).Valor_Fechamento[1:int(len70)+1]
y_test2 = dfCasoDeEstudo.fillna(0).Valor_Fechamento[int(len70)+1:maxlen]
df2019_train2 = df2019_train2.fillna(0)
df2019_test2 = df2019_test2.fillna(0)
print(f"df2019_train shape is {df2019_train2.shape}")
print(f"df2019_test shape is {df2019_test2.shape}")
print(f"y_train shape is {y_train2.shape}")
print(f"y_test shape is {y_test2.shape}")

In [ ]:
batch_size = 32
time_steps = 1
prediction = 1 #valor a ser previsto
num_features = 4 # numero de atributos usados; Preco Medio BTC, Valor_Abertura, Valor_Minima, Valor_Maxima

In [ ]:
RS_VALUE = True	
K.backend.clear_session()
tf.logging.set_verbosity(tf.logging.ERROR) #desliga os warnings do tensorflow
init   = K.initializers.glorot_uniform(seed=1)
theOptimizer = K.optimizers.RMSprop()
model2 = K.models.Sequential()
model2.add(K.layers.LSTM(units=32, return_sequences=RS_VALUE,
                        batch_input_shape=(batch_size, time_steps, num_features),
                        kernel_initializer=init, activation='sigmoid'))
model2.add(K.layers.LSTM(units=8, return_sequences=RS_VALUE))
model2.add(K.layers.LSTM(units=5, return_sequences=RS_VALUE))
model2.add(K.layers.Dense(units=1,  kernel_initializer=init, activation='relu'))
#model2.compile(loss='mean_squared_error', optimizer=theOptimizer, metrics=['mae','mse'])
model2.compile(optimizer=theOptimizer,
              loss=tf.keras.metrics.mean_squared_error,
              metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
model2.summary()

In [ ]:
X2 = df2019_train2[0:((df2019_train2.Preco_Medio_BTC.size) - (df2019_train2.Preco_Medio_BTC.size % time_steps))]
X_Train2 = np.zeros((num_features,time_steps,df2019_train2.Preco_Medio_BTC.size))
y2 = y_train2.values[0:((df2019_train2.Preco_Medio_BTC.size) - (df2019_train2.Preco_Medio_BTC.size % time_steps))]
trainY2 = y2.reshape(-1, time_steps, 1)

In [ ]:
X_Train2[0][0] = X2.Valor_Abertura
X_Train2[1][0] = X2.Valor_Minima
X_Train2[2][0] = X2.Valor_Maxima
X_Train2[3][0] = X2.Preco_Medio_BTC

In [ ]:
#Train	
print("Iniciando treinamento... ")
h = model2.fit(X_Train2.T, trainY2, batch_size=batch_size, shuffle=False, verbose=1)
print("Treinamento finalizado \n")


In [ ]:
dftest2 = df2019_test2[0:((df2019_test2.Preco_Medio_BTC.size) - (df2019_test2.Preco_Medio_BTC.size % time_steps))]
X_test2 = np.zeros((num_features,time_steps,df2019_test2.Preco_Medio_BTC.size))
X_test2[0][0] = dftest2.Valor_Abertura
X_test2[1][0] = dftest2.Valor_Minima
X_test2[2][0] = dftest2.Valor_Maxima
X_test2[3][0] = dftest2.Preco_Medio_BTC
seriesY2 = y_test2.values[0:((df2019_test2.Preco_Medio_BTC.size) - (df2019_test2.Preco_Medio_BTC.size % time_steps))]
testY2 = seriesY2.reshape(-1, time_steps, 1)

In [ ]:
eval = model2.evaluate(X_test2.T, testY2, verbose=0)
print("Dados de teste: loss = %0.6f MAE = %0.2f%% \n" \
% (eval[0], eval[1]*100) )

In [ ]:
#pegar valores
y_pred2 = model2.predict(X_test2.T, batch_size=batch_size, verbose=0)

In [ ]:
y_pred2[0]

# Conclusão

### Infelizmente não foi possível adquirir um resultado confiável com base na rede utilizada, em ambos os treinamentos

Inicialmente, foi feito um treinamento com dados de saida normalizados, e em seguida um com o valores puros e mais variaveis de entrada.
O primeiro treinamento apresentou MAE de aproximadamente 50%, enquanto O segundo retornou um MAE de 70%, ambos valores muito altos para o erro absoluto

Contudo, o segundo treinamento conseguiu retornar um valor de predição, que era nosso objetivo inicial. Esse valor calculado seria o Valor de Fechamento do Bitcoin calculado para o dia seguinte da amostra. Infelizmente com uma acurácia tão baixa ainda não é possivel utilizar esse valor como dado real para investimento, mas já é um resultado que pode ser tunado e otimizado com o tempo. 

Assim, a conclusão que podemos obter segue a apresentada acima com as questões. O valor de bitcoin aparenta ser muito volátil para identificar ele mesmo com uma rede neural, e esta volatilidade impede que um potencial investidor possa tomar decisões com precisão melhor do que só jogar uma moeda e decidir pela sorte. Além disso, as margens que podemos conseguir no Day Trade são tão baixas que não valem o esforço de desenvolver um algoritmo que consiga operar nessa escala, valendo mais a pena focar no Swing Trade e obter lucros "reais".

### Pos-Conclusão, treinamentos adicionais


Apos a conclusão, decidimos utilizar ao invez de LSTM uma abordagem mais simples, com uma regressão linear, para observar as diferencas nos resultados obtidos.

In [ ]:
#Treinamento como regressão
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import datasets
import matplotlib.pylab as plt

In [ ]:
from sklearn.model_selection import train_test_split
X_train_sk, X_test_sk, y_train_sk, y_test_sk = train_test_split(dfCasoDeEstudo.reset_index().fillna(0).drop(['index','DataHora','Valor_Fechamento', 'BTC_Negociado', 'USD_Negociado'], axis=1),
                                                   dfCasoDeEstudo.fillna(0).Valor_Fechamento, test_size=0.33, random_state=0)

In [ ]:
model_sk = LinearRegression()
# 2. Use fit
model_sk.fit(X_train_sk, y_train_sk)
# 3. Check the score
model_sk.score(X_test_sk, y_test_sk)

In [ ]:
model_sk.predict(X_test_sk)

In [ ]:
y_pred_sk = model_sk.predict(X_test_sk) 
plt.plot(y_test_sk, y_pred_sk, '.')

# plot a line, a perfit predict would all fall on this line
x = np.linspace(0, 330, 100)
y = x
plt.plot(x, y)
plt.show()

In [ ]:
model_sk_2 = LinearRegression()
model_sk_2.fit(df2019_train2, y_train2)
model_sk_2.score(df2019_test2, y_test2)

In [ ]:
y_pred_sk = model_sk_2.predict(df2019_test2) 
plt.plot(y_test2, y_pred_sk, '.')
x = np.linspace(0, 330, 100)
y = x
plt.plot(x, y)
plt.show()

In [ ]:
dfjun2019 = dfBitstamp[dfBitstamp.DataHora.dt.year >= 2019]
dfjun2019 = dfjun2019[dfjun2019.DataHora.dt.month >= 6]
dfjun2019

In [ ]:
X_train_jun, X_test_jun, y_train_jun, y_test_jun = train_test_split(dfjun2019.reset_index().fillna(0).drop(['index','DataHora','Valor_Fechamento', 'BTC_Negociado', 'USD_Negociado', 'Preco_Medio_BTC'], axis=1),
                                                   dfjun2019.fillna(0).Valor_Fechamento, test_size=0.33, random_state=0)

In [ ]:
model_jun = LinearRegression()
# 2. Use fit
model_jun.fit(X_train_jun, y_train_jun)
# 3. Check the score
model_jun.score(X_test_jun, y_test_jun)

In [ ]:
model_jun.predict(X_test_jun)

In [ ]:
y_pred_jun = model_jun.predict(X_test_jun) 
plt.plot(y_test_jun, y_pred_jun, '.')
x = np.linspace(0, 330, 100)
y = x
plt.plot(x, y)
plt.show()